In [1]:
ruta_data = r'G:'

In [2]:
path_xlsx_parte = ruta_data + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Pulverizacion\2025\parte_diario_pulverizacion.xlsx'

In [3]:
import sys
sys.path.append('../_amigocloud')
from amigocloud import AmigoCloud
import pandas as pd
from shapely import wkb
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon, MultiPoint
import os

C:\Users\Usuario\AppData\Local\Temp\ipykernel_31940\4047952240.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

In [5]:
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [6]:
consulta = {'query': 'select * from dataset_367119'}
seleccion = amigocloud.get('https://app.amigocloud.com/api/v1/projects/35248/sql', consulta)
datos = seleccion['data']

In [7]:
df_datos = gpd.GeoDataFrame(datos)

In [8]:
df_datos

,id,amigo_id,fecha_registro,referencia,temp_max,temp_min,viento,humedad,observaciones,tipo_cultivo,tipo_desarrollo
0,15,fb817034f73b47ecbeb103c76cae3389,2025-03-19 15:23:33+00:00,0101000020E6100000B7E0FBD1269A4FC0645930F1473D...,26.0,26.0,5.0,75.0,None,CAÑA DE AZUCAR,CRECIMIENTO


In [9]:
list(set(df_datos['id']))

[15]

In [10]:
idd = 15

In [11]:
query = {'query': f'SELECT apli.*, plan.unidad_01, plan.unidad_03, plan.os\
    FROM dataset_367119 apli\
    JOIN dataset_360912 plan\
    ON ST_Intersects(apli.referencia, plan.geometry)\
    where apli.id = {idd}'}
select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/35248/sql', query)
data_apli = select['data']
#df_apli = pd.DataFrame(data_apli)
#df_apli

In [12]:
dic_apli = data_apli[0]
dic_apli

{'id': 15,
 'amigo_id': 'fb817034f73b47ecbeb103c76cae3389',
 'fecha_registro': '2025-03-19 15:23:33+00:00',
 'referencia': '0101000020E6100000B7E0FBD1269A4FC0645930F1473D31C0',
 'temp_max': 26.0,
 'temp_min': 26.0,
 'viento': 5.0,
 'humedad': 75.0,
 'observaciones': None,
 'tipo_cultivo': 'CAÑA DE AZUCAR',
 'tipo_desarrollo': 'CRECIMIENTO',
 'unidad_01': 299,
 'unidad_03': 1530,
 'os': 5}

In [13]:
cod_prop = dic_apli['unidad_01']
cod_ca = dic_apli['unidad_03']
oss = dic_apli['os']

In [14]:
print('propiedad:', cod_prop)
print('cañero:', cod_ca)
print('os:', oss)

propiedad: 299
cañero: 1530
os: 5


In [15]:
#nulos = dic_apli.isnull().sum()
#nulos

In [16]:
df_parte = pd.read_excel(path_xlsx_parte)
df_parte.head(3)

,Nº DE LABOR,FECHA,SEMANA,INSTITUCION,COD CANERO,NOMBRE CANERO,COD PROPIEDAD,NOMBRE PROPIEDAD,LOTE,HECTAREA,...,OS,TEMP MAX,TEMP MIN,TIPO CULTIVO,TIPO DESARROLLO,NUM VUELOS,HORAS VUELO,TOTAL CAUDAL,idd,AREA PULVERIZADO
0,PA-2025-1,2025-03-07,10.0,50,41594,AGROPECUARIA CAMPO DULCE S.R.L.,30,CAMPO DULCE,EP-L31,30.260942,...,1,25.0,20.0,CAÑA DE AZUCAR,MADURACION,104.0,15:03:42,2952.664,1.0,28.633271
1,PA-2025-1,2025-03-07,10.0,50,41594,AGROPECUARIA CAMPO DULCE S.R.L.,30,CAMPO DULCE,EP-L16,23.216246,...,1,25.0,20.0,CAÑA DE AZUCAR,MADURACION,104.0,15:03:42,2952.664,1.0,21.664272
2,PA-2025-1,2025-03-07,10.0,50,41594,AGROPECUARIA CAMPO DULCE S.R.L.,30,CAMPO DULCE,ER-L6,19.683505,...,1,25.0,20.0,CAÑA DE AZUCAR,MADURACION,104.0,15:03:42,2952.664,1.0,19.160233


In [17]:
df_parte['FECHA'] = pd.to_datetime(df_parte['FECHA'], errors='coerce')
df_parte['HORA DE INICIO'] = pd.to_datetime(df_parte['HORA DE INICIO'], errors='coerce')
df_parte['HORA FINAL'] = pd.to_datetime(df_parte['HORA FINAL'], errors='coerce')

In [18]:
df_parte.loc[(df_parte['COD PROPIEDAD'] == dic_apli['unidad_01']) & 
             (df_parte['COD CANERO'] == dic_apli['unidad_03']) & 
             (df_parte['OS'] == dic_apli['os']), 'idd'] = dic_apli['id']
df_parte.loc[(df_parte['COD PROPIEDAD'] == dic_apli['unidad_01']) & 
             (df_parte['COD CANERO'] == dic_apli['unidad_03']) & 
             (df_parte['OS'] == dic_apli['os']), 'TEMP MAX'] = dic_apli['temp_max']
df_parte.loc[(df_parte['COD PROPIEDAD'] == dic_apli['unidad_01']) & 
             (df_parte['COD CANERO'] == dic_apli['unidad_03']) & 
             (df_parte['OS'] == dic_apli['os']), 'TEMP MIN'] = dic_apli['temp_min']
df_parte.loc[(df_parte['COD PROPIEDAD'] == dic_apli['unidad_01']) & 
             (df_parte['COD CANERO'] == dic_apli['unidad_03']) & 
             (df_parte['OS'] == dic_apli['os']), 'VIENTO'] = dic_apli['viento']
df_parte.loc[(df_parte['COD PROPIEDAD'] == dic_apli['unidad_01']) & 
             (df_parte['COD CANERO'] == dic_apli['unidad_03']) & 
             (df_parte['OS'] == dic_apli['os']), 'HUMEDAD'] = dic_apli['humedad']
df_parte.loc[(df_parte['COD PROPIEDAD'] == dic_apli['unidad_01']) & 
             (df_parte['COD CANERO'] == dic_apli['unidad_03']) & 
             (df_parte['OS'] == dic_apli['os']), 'TIPO CULTIVO'] = dic_apli['tipo_cultivo']
df_parte.loc[(df_parte['COD PROPIEDAD'] == dic_apli['unidad_01']) & 
             (df_parte['COD CANERO'] == dic_apli['unidad_03']) & 
             (df_parte['OS'] == dic_apli['os']), 'TIPO DESARROLLO'] = dic_apli['tipo_desarrollo']

In [19]:
df_parte.head(3)

,Nº DE LABOR,FECHA,SEMANA,INSTITUCION,COD CANERO,NOMBRE CANERO,COD PROPIEDAD,NOMBRE PROPIEDAD,LOTE,HECTAREA,...,OS,TEMP MAX,TEMP MIN,TIPO CULTIVO,TIPO DESARROLLO,NUM VUELOS,HORAS VUELO,TOTAL CAUDAL,idd,AREA PULVERIZADO
0,PA-2025-1,2025-03-07,10.0,50,41594,AGROPECUARIA CAMPO DULCE S.R.L.,30,CAMPO DULCE,EP-L31,30.260942,...,1,25.0,20.0,CAÑA DE AZUCAR,MADURACION,104.0,15:03:42,2952.664,1.0,28.633271
1,PA-2025-1,2025-03-07,10.0,50,41594,AGROPECUARIA CAMPO DULCE S.R.L.,30,CAMPO DULCE,EP-L16,23.216246,...,1,25.0,20.0,CAÑA DE AZUCAR,MADURACION,104.0,15:03:42,2952.664,1.0,21.664272
2,PA-2025-1,2025-03-07,10.0,50,41594,AGROPECUARIA CAMPO DULCE S.R.L.,30,CAMPO DULCE,ER-L6,19.683505,...,1,25.0,20.0,CAÑA DE AZUCAR,MADURACION,104.0,15:03:42,2952.664,1.0,19.160233


In [20]:
output_path = path_xlsx_parte
df_parte.to_excel(output_path, index=False)

# HACER COPIA Y ELIMINAR PARTE DIARIO Y PLANIFICACION DE AMIGOCLOUD

In [21]:
idd

15

In [22]:
#copia y elimina los idd indicados del parte diario de amigocloud
id_proyecto = f'https://app.amigocloud.com/api/v1/projects/35248/sql'
query = f'INSERT INTO dataset_367584 (id, fecha_registro, temp_max, temp_min, viento, humedad, observaciones, tipo_cultivo, tipo_desarrollo, referencia)\
        SELECT id, fecha_registro, temp_max, temp_min, viento, humedad, observaciones, tipo_cultivo, tipo_desarrollo, referencia\
        FROM dataset_367119\
        WHERE id = {idd}'
query_sql = {'query': query}
post = amigocloud.post(id_proyecto, query_sql)
query = {'query': f'DELETE FROM dataset_367119 WHERE id = {idd}'}
ejecutar = amigocloud.post(id_proyecto, query)
print('se proceso IDD:', idd)

se proceso IDD: 15


In [23]:
#copia y elimina los id indicados de la planificacion semanal de amigocloud
id_proyecto = f'https://app.amigocloud.com/api/v1/projects/35248/sql'
query = f'INSERT INTO dataset_360915 (fecha_registro, id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, os, geometry)\
            SELECT fecha_registro, id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, os, geometry\
            FROM dataset_360912\
            WHERE unidad_01 = {cod_prop} and os = {oss}'
query_sql = {'query': query}
post = amigocloud.post(id_proyecto, query_sql)
query = {'query': f'DELETE FROM dataset_360912 WHERE unidad_01 = {cod_prop} and os = {oss}'}
ejecutar = amigocloud.post(id_proyecto, query)
print('se proceso ID:', idd)

se proceso ID: 15


## Copiar ids de PARTE DIARIO a PLANIFICACION

In [24]:
path_planificacion = ruta_data + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Pulverizacion\2025\SHP\PLANIFICACION.shp'

In [25]:
gdf_plan = gpd.read_file(path_planificacion)

In [26]:
gdf_plan.loc[(gdf_plan['unidad_01'] == dic_apli['unidad_01']) & 
             (gdf_plan['unidad_03'] == dic_apli['unidad_03']) & 
             (gdf_plan['os'] == dic_apli['os']), 'idd'] = dic_apli['id']

In [27]:
gdf_plan

,unidad_01,unidad_02,unidad_03,unidad_04,unidad_05,area,os,soca,id,inst,idd,geometry
0,30,CAMPO DULCE,41594,AGROPECUARIA CAMPO DULCE S.R.L.,EP-L31,30.260942,1,2,1,50,1.0,"POLYGON ((504085.659 8094406.031, 503737.808 8..."
1,30,CAMPO DULCE,41594,AGROPECUARIA CAMPO DULCE S.R.L.,EP-L16,23.216246,1,4,2,50,1.0,"POLYGON ((504352.053 8094797.970, 504272.088 8..."
2,30,CAMPO DULCE,41594,AGROPECUARIA CAMPO DULCE S.R.L.,ER-L6,19.683505,1,5,3,50,1.0,"POLYGON ((503260.583 8094596.306, 503714.735 8..."
3,30,CAMPO DULCE,41594,AGROPECUARIA CAMPO DULCE S.R.L.,EP-L35,7.795719,1,4,4,50,1.0,"POLYGON ((504693.348 8094607.629, 504673.327 8..."
4,30,CAMPO DULCE,41594,AGROPECUARIA CAMPO DULCE S.R.L.,EP-L34,7.048921,1,5,5,50,1.0,"POLYGON ((504137.327 8094415.183, 504648.080 8..."
...,...,...,...,...,...,...,...,...,...,...,...,...
92,299,CITTCA,1530,C.I.T.T.C.A.,C20.2,3.812829,7,1,98,124,NaN,"POLYGON ((480706.761 8093557.478, 480708.890 8..."
93,299,CITTCA,1530,C.I.T.T.C.A.,N15.1,0.616534,7,4,99,124,NaN,"POLYGON ((479030.491 8094202.164, 479030.045 8..."
94,299,CITTCA,1530,C.I.T.T.C.A.,N14.1,0.457476,7,1,100,124,NaN,"POLYGON ((479018.402 8094327.888, 479019.454 8..."
95,299,CITTCA,1530,C.I.T.T.C.A.,N6,4.689468,7,0,101,124,NaN,"POLYGON ((478582.154 8094337.075, 478582.813 8..."


In [28]:
gdf_plan.dtypes

unidad_01       int64
unidad_02      object
unidad_03       int64
unidad_04      object
unidad_05      object
area          float64
os              int64
soca            int64
id              int64
inst            int64
idd           float64
geometry     geometry
dtype: object

In [29]:
gdf_plan.to_file(path_planificacion, driver='ESRI Shapefile')